# Bagging: Estratégia de Ensemble Learning para Modelos Preditivos

---

## **Passo a Passo para o Bagging**

O **Bagging** (Bootstrap Aggregating) é uma técnica de aprendizado de máquina baseada no conceito de *ensemble learning*, onde múltiplos modelos são treinados de forma independente e suas previsões são combinadas para obter um resultado mais robusto e confiável. O processo pode ser dividido em três etapas principais:

### **🔹 Etapa 1: Bootstrap (Criação de Subamostras)**
Nesta fase, múltiplas amostras de treinamento são geradas a partir do conjunto de dados original por meio de um processo conhecido como *bootstrap sampling*. Esse método envolve **amostragem com reposição**, ou seja, alguns exemplos podem aparecer várias vezes em uma amostra, enquanto outros podem não ser incluídos. O tamanho de cada amostra bootstrap é igual ao conjunto de dados original.

### **🔹 Etapa 2: Modelagem (Aprendizes de Base)**
Cada amostra bootstrap é utilizada para treinar um **modelo de aprendizado de máquina independente**. Os modelos podem ser árvores de decisão, redes neurais ou qualquer outro algoritmo preditivo. Como cada modelo é treinado com um subconjunto diferente dos dados, ele captura diferentes aspectos do padrão presente no conjunto de treinamento.

### **🔹 Etapa 3: Agregação das Previsões**
Após o treinamento, as previsões dos modelos individuais são combinadas para formar uma única previsão final:
- **Para tarefas de classificação**, utiliza-se a **votação majoritária**, onde a classe mais predita entre os modelos é selecionada.
- **Para tarefas de regressão**, a **média das previsões** dos modelos individuais é utilizada como a estimativa final.

O Bagging reduz a **variância do modelo**, tornando as previsões mais estáveis e menos sensíveis a flutuações nos dados, sem um aumento significativo do viés.

---

## **Explicação Conceitual do Bagging**

O **Bagging** é uma técnica de *ensemble learning* que aprimora a estabilidade e a precisão de modelos preditivos ao reduzir a variabilidade nos resultados. O princípio central do Bagging é gerar **diversidade** entre os modelos de aprendizado, de modo que suas previsões individuais possam ser combinadas para formar um resultado mais confiável.

O processo se inicia com a geração de diversas **amostras bootstrap**, que são subconjuntos do conjunto de dados original obtidos por amostragem com reposição. Cada uma dessas amostras serve como base para o treinamento de um **modelo individual**. Como cada modelo aprende um conjunto ligeiramente diferente dos dados, eles se tornam menos propensos ao *overfitting* (superajuste).

Após o treinamento, os modelos são testados sobre um mesmo conjunto de dados e suas previsões são **agregadas** de forma estratégica:
- **Para classificação**, a **votação majoritária** é aplicada, selecionando a classe mais recorrente entre os modelos.
- **Para regressão**, a **média das previsões** é utilizada como a resposta final.

O Bagging é amplamente utilizado em algoritmos como **Random Forest**, onde diversas árvores de decisão são treinadas em subconjuntos dos dados e agregadas para melhorar a performance geral. Essa abordagem confere maior **robustez, estabilidade e generalização**, tornando-a altamente eficaz em problemas complexos.

---

## **Implementação em Python do Bagging**

A seguir, apresentamos a implementação do **Bagging** para **classificação** e **regressão**, demonstrando na prática como essa técnica pode ser aplicada utilizando o **Scikit-learn**.

### **Bagging para Classificação**
A implementação a seguir utiliza um **conjunto de dados de classificação (Iris Dataset)** para ilustrar o uso de árvores de decisão como aprendizes de base dentro da estratégia de Bagging.

In [14]:
# Import das bibliotecas

import pandas as pd

from sklearn.datasets        import load_iris
from sklearn.datasets        import load_diabetes

from sklearn.model_selection import train_test_split

from sklearn.tree            import DecisionTreeClassifier
from sklearn.metrics         import accuracy_score

from sklearn.tree            import DecisionTreeRegressor
from sklearn.metrics         import mean_squared_error, r2_score

In [16]:
# Exemplo da técnica de bagging para problemas de classificação

X = load_iris().data
y = load_iris().target

df = pd.DataFrame(X, columns=load_iris().feature_names)
df['target'] = y

def bagging_classifier(df:pd.DataFrame, 
                       num_bootstrap_samples:int=3,  # Parâmetro da função que define a quantidade de amostragens para treinamento
                       test_size:float=0.25
                      ) -> pd.DataFrame:
    
    df_train, df_test = train_test_split(df, test_size=test_size)
    
    X_test = df_test.drop(['target'], axis=1)
    y_test = df_test['target'].rename('y_test')
    
    # Dicionário para os resultados das predições de cada modelo
    y_pred_bagging = {}

    for i in range(num_bootstrap_samples):
        # Bootstrap
        df_train = df_train.sample(n=len(df_train), 
                                   replace=True)  # Amostragem COM reposição

        X_train = df_train.drop(['target'], axis=1)
        y_train = df_train['target']
        
        # Modelagem (base learners)
        model = DecisionTreeClassifier()
        model.fit(X_train, y_train)
        
        # Adicionando os resultados do modelo ao dicionário para agregação das predições
        y_pred_bagging.update({i:model.predict(X_test)})
    
    # Aggregating
    y_pred = (pd.DataFrame(y_pred_bagging)
                .mode(axis=1)  # Agregando o valor com maior número de aparições nas predições dos modelos
                .rename(columns={0:'y_pred'}))
 
    # Resultados
    print(model)
    print('Accuracy score:', accuracy_score(y_true=y_test, 
                                            y_pred=y_pred['y_pred']
                                           ))

    return pd.concat(objs=[y_test.reset_index(drop=True), 
                           y_pred['y_pred'].astype(int)], 
                     axis=1)

bagging_classifier(num_bootstrap_samples=10, df=df, test_size=0.33)

DecisionTreeClassifier()
Accuracy score: 0.96


,y_test,y_pred
0,2,2
1,1,1
2,2,2
3,2,2
4,0,0
5,1,1
6,1,1
7,2,2
8,2,2
9,0,0


### **Bagging para Regressão**
Agora, aplicamos o Bagging ao **conjunto de dados de regressão (Diabetes Dataset)**, utilizando **árvores de decisão regressivas** como aprendizes de base.

In [3]:
# Exemplo da técnica de bagging para problemas de regressão

X = load_diabetes().data
y = load_diabetes().target

df = pd.DataFrame(X, columns=load_diabetes().feature_names)
df['target'] = y

def bagging_regressor(df:pd.DataFrame, 
                      num_bootstrap_samples:int=3,  # Parâmetro da função que define a quantidade de amostragens para treinamento
                      test_size:float=0.25
                      ) -> pd.DataFrame:
    
    df_train, df_test = train_test_split(df, test_size=test_size)
    
    X_test = df_test.drop(['target'], axis=1)
    y_test = df_test['target'].rename('y_test')
    
    # Dicionário para os resultados das predições de cada modelo
    y_pred_bagging = {}

    for i in range(num_bootstrap_samples):
        # Bootstrap
        df_train = df_train.sample(n=len(df_train), 
                                   replace=True)  # Amostragem COM reposição

        X_train = df_train.drop(['target'], axis=1)
        y_train = df_train['target']

        # Modelagem (base learners)
        model = DecisionTreeRegressor()
        model.fit(X_train, y_train)
        
        # Adicionando os resultados do modelo ao dicionário para agregação das predições
        y_pred_bagging.update({i:model.predict(X_test)})

    # Aggregating
    y_pred = (pd.DataFrame(y_pred_bagging)
                .mean(axis=1)  # Agregando as predições dos modelos baseando n a média dos resultados
                .rename('y_pred'))
 
    # Resultados
    print(model)
    print('Mean squared error:', mean_squared_error(y_true=y_test, 
                                                   y_pred=y_pred))
    print('Coefficient of determination:', r2_score(y_true=y_test, 
                                                    y_pred=y_pred))
    
    return pd.concat(objs=[y_test.reset_index(drop=True), 
                           y_pred], 
                     axis=1)
    
bagging_regressor(num_bootstrap_samples=100, df=df, test_size=0.33)

DecisionTreeRegressor()
Mean squared error: 3495.8629835616434
Coefficient of determination: 0.3819796763415946


,y_test,y_pred
0,246.0,186.49
1,248.0,203.83
2,182.0,117.78
3,142.0,184.31
4,154.0,136.21
...,...,...
141,172.0,180.12
142,185.0,137.12
143,67.0,159.76
144,97.0,98.48


---

### **Conclusão**

A aplicação da técnica de **Bagging** demonstrou sua eficácia tanto em problemas de classificação quanto em regressão, utilizando árvores de decisão como base learners.

Para **classificação**, o modelo obteve uma **acurácia de 92%**, indicando que a agregação de múltiplos modelos através do Bagging contribuiu significativamente para a melhoria da estabilidade e da precisão preditiva. O uso de amostragem bootstrap garantiu diversidade nos dados de treinamento, reduzindo o risco de overfitting e aumentando a generalização do modelo.

Já no caso da **regressão**, a métrica de erro quadrático médio (**MSE**) foi **3495,86**, e o coeficiente de determinação (**R²**) atingiu **0,38**, sugerindo que, embora o modelo tenha conseguido capturar parte da variação dos dados, há espaço para melhorias, como a otimização de hiperparâmetros ou a combinação com técnicas mais robustas, como Random Forest ou Boosting.